In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

import fiona
import folium
from shapely import wkt
from shapely.geometry import shape
import ast

import json
from loguru import logger
from pathlib import Path

from geowrangler.datasets import geofabrik

import sys

sys.path.append("../../../")  # include parent directory
from src.vector_utils import *

/home/abbymoreno/miniconda3/envs/lacuna-fund2/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
# data directories
DATA_DIR = Path("../../../data/")
SRC_DIR = Path("../../../src/")
ADMIN_FPATH = DATA_DIR / "01-admin-bounds"
RAW_FPATH = DATA_DIR / "02-raw"
PROCESSED_FPATH = DATA_DIR / "03-processed"
OUTPUT_FPATH = DATA_DIR / "04-output"
GIS_FPATH = DATA_DIR / "05-gis"

### Load administrative boundaries

In [5]:
# Load the administrative boundaries for the 12 cities
admin_bounds_gdf = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
admin_bounds_gdf.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


### Load coastal buffer

In [9]:
# Load coastal buffer generated from the previous notebook.
coast_buffer = gpd.read_file(OUTPUT_FPATH / "ph_coasts_30m.gpkg")
coast_buffer.head(2)

,uid,geometry
0,0,"POLYGON ((119.20565 4.66970, 119.20750 4.66894..."
1,1,"POLYGON ((119.03071 4.70988, 119.03140 4.70875..."


# Get OSM Water features

Water features:
- distance to nearest body of water (man-made or natural)
- check if near the coast or not

In [8]:
osm_water_feats_gdf = add_osm_water_features(
    admin_bounds_gdf,
    "philippines",
    "2022",
)
osm_water_feats_gdf.head(2)

2023-05-29 18:12:43.870 | DEBUG    | src.vector_utils:add_osm_water_features:184 - OSM Water for philippines and year 2022 being loaded from ../../../data/02-raw/osm/philippines-220101-free.shp.zip


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,osm_year,osm_wetland_nearest,osm_reservoir_nearest,osm_water_nearest,osm_riverbank_nearest,osm_dock_nearest
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",2022,1601.942768,10000.0,0.0,0.0,10000.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2022,0.000000,10000.0,0.0,0.0,10000.0


In [9]:
osm_waterways_feats_gdf = add_osm_water_features(
    admin_bounds_gdf,
    "philippines",
    "2022",
    waterways=True,
)
osm_waterways_feats_gdf.head(2)

2023-05-29 18:20:14.173 | DEBUG    | src.vector_utils:add_osm_water_features:184 - OSM Water for philippines and year 2022 being loaded from ../../../data/02-raw/osm/philippines-220101-free.shp.zip


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,osm_year,osm_river_nearest,osm_stream_nearest,osm_canal_nearest,osm_drain_nearest
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",2022,0.0,2102.032556,1560.588579,2236.652574
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2022,0.0,105.478349,149.797580,761.117734


# Get Distance to Coast

In [10]:
%%time
coast_feat = add_distance_to_shore(
    admin_bounds_gdf, coastal_buffer_path=OUTPUT_FPATH / "ph_coasts_30m.gpkg"
)
coast_feat.head(2)

CPU times: user 11min 57s, sys: 302 ms, total: 11min 57s
Wall time: 11min 57s


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,distance_from_coast
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",308.382889
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2595.485717


In [11]:
# label coastal barangays
coast_feat.loc[(coast_feat["distance_from_coast"] == 0), "is_coastal"] = True
coast_feat.loc[(coast_feat["distance_from_coast"] != 0), "is_coastal"] = False

In [12]:
# map of poi counts
GOOGLE_BASEMAP = folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
    overlay=True,
    control=True,
    show=True,
)

coast_base = coast_buffer.explore(color="white")

coast_feat.explore(column="is_coastal", m=coast_base)

GOOGLE_BASEMAP.add_to(coast_base)

# UNCOMMENT TO DISPLAY MAP
# coast_base

In [ ]:
# UNCOMMENT TO SAVE
# coast_base.save(GIS_FPATH / "inspect_coastal_barangays_30m.html")